In [2]:
import pandas as pd
from pprint import pprint
import glob
import pandas as pd
import random
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from config import event_types
random.seed(42)
pd.set_option('display.max_rows', None)
# jupyter nbconvert --to script 04b_combine_results_find_disagreement.ipynb
# sudo kill -9 $(nvidia-smi | awk 'NR>8 {print $5}' | grep -E '^[0-9]+$')
version = 1
from config import event_types

for model_type in ["dictionary", "biolord"]:
    output_folder = f"../exports/selected_reports_with_event_log_only_{model_type}_v{version}"
    output_file = f"{output_folder}/combined.pkl"
    batch_files = sorted(glob.glob(f"{output_folder}/batch_*.pkl"))
    combined_df = pd.concat([pd.read_pickle(f) for f in batch_files], ignore_index=True)
    combined_df.to_pickle(output_file)
   

def prepare_df(df, type="biolord"):
    global event_types
    df = df.copy()
    print("Loaded file len: ",len(df))
    df['Sent_ID'] = df['Events'].apply(lambda x: [f"{i:04d}" for i in range(len(x))])
    df = df.explode(["Sent_ID","Events"])
    print("After exploding file len: ",len(df))
    df['UID'] = df['ROW_ID'].astype(str) + "_" + df['Sent_ID'].astype(str)
    df = df.dropna(subset="Events")
    df['Event_Name'] = df['Events'].apply(lambda x: x['event'])
    df['Sentence'] = df['Events'].apply(lambda x: x['sentence'])
    df['Time'] = df['Events'].apply(lambda x: x['event_detection_time'])
    
    if type == "dictionary":
        df['Keyword'] = df['Events'].apply(lambda x: x['keyword'])
        df['Lemma'] = df['Events'].apply(lambda x: x['lemma'])
    if type == "biolord":
        df['Similarity'] = df['Events'].apply(lambda x: x['similarity'])
        df['Similarity'] = df['Similarity'].apply(lambda x: {k:v for (k,v) in x.items() if k!="Alert And Oriented"})
        for ET in event_types:
            df[f"{ET}_similarity"] = df['Similarity'].apply(lambda x:x[ET])
            
    return df
df_dictionary = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_dictionary_v{version}/combined.pkl"),type="dictionary")
df_biolord = prepare_df(pd.read_pickle(f"../exports/selected_reports_with_event_log_only_biolord_v{version}/combined.pkl"), type = "biolord")






Loaded file len:  15222
After exploding file len:  338395
Loaded file len:  15222
After exploding file len:  338395


In [3]:
# df_biolord.Similarity = df_biolord.Similarity.astype(str)
similarity_col_names = [f"{ET}_similarity" for ET in event_types]
df_both = pd.merge(df_dictionary[['UID','ROW_ID','Sent_ID','HADM_ID','CHARTTIME','STORETIME','Sentence','Event_Name','Keyword','Lemma','CGID','Time']], 
         df_biolord[['UID','ROW_ID','Sent_ID','HADM_ID','CHARTTIME','STORETIME','Sentence','Event_Name','CGID','Similarity','Time']+similarity_col_names], 
         on=['UID','HADM_ID','ROW_ID','Sent_ID'], how='outer',suffixes=("_dictionary","_biolord")).sort_values(by=['HADM_ID','ROW_ID','Sent_ID'])
df_both["Event_Name_dictionary_multi"] = df_both["Event_Name_dictionary"].apply(lambda x: {i: x.count(i) if isinstance(x, list) else 0 for i in event_types})
len(df_dictionary), len(df_biolord), len(df_both)
df_both.to_pickle("../exports/dictionary_biolord_features.pkl")

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import LinearSegmentedColormap

# Define a custom color gradient (e.g., black → red → yellow → white)
custom_cmap = LinearSegmentedColormap.from_list(
    name='custom_gradient',
    colors=["skyblue", "yellow"]
)
def plot_axis(counts, i, ax, column, num_bins, bin_edges):

    norm_counts = counts / counts.sum() * 100

    sns.heatmap(
    [norm_counts],
    ax=ax,
    cmap=custom_cmap,
    cbar=False,
    xticklabels=False if i < len(similarity_columns) - 1 else True,
    yticklabels=False,
    annot=[norm_counts],                 # Annotations need to match the shape of data (1 row)
    fmt='0.1f',                        # Integer format
    annot_kws={"size": 6, "color": "black", "rotation": 90, "va": "center", "ha": "center"}  # Styling
)
    if i%2==0:
        pref = "W"
    else:
        pref = "W/o"
    # Label each heatmap
    ax.set_ylabel(f"{pref} {column} KW \n Total:{counts.sum()}", rotation=0, labelpad=50, va='center')
    ax.set_yticks([])

    # Only format x-ticks for the bottom axis
    if i == 9:
        # Compute bin centers
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

        # Choose a few bin positions to label
        tick_indices = np.linspace(0, num_bins - 1, 25).astype(int)
        tick_labels = [f"{bin_centers[j]:.2f}" for j in tick_indices]

        # Now set ticks and labels
        ax.set_xticks(tick_indices)
        ax.set_xticklabels(tick_labels, rotation=90, ha="right")

# Similarity columns to plot
similarity_columns = event_types

# Compute global min and max for consistent binning
all_values = np.concatenate([df_both[f"{col}_similarity"].dropna().values for col in similarity_columns])
global_min, global_max = all_values.min(), all_values.max()

# Define consistent bins
num_bins = 50
bin_edges = np.linspace(global_min, global_max, num_bins + 1)

# Create subplots
fig, axes = plt.subplots(
    nrows=len(similarity_columns)*2,
    figsize=(12, 8),
    sharex=True
)

for i, ax in enumerate(axes):
    column = similarity_columns[i//2]
    event_type = column
    # if i%2 == 0:
    #     df_sel = df_both[df_both["Event_Name_dictionary"].apply(lambda x: "_".join(set(x)) == event_type )]
    # else:
    #     df_sel = df_both[df_both["Event_Name_dictionary"].apply(lambda x: event_type not in x and "_".join(set(x))!='Unknown' )]
    if i%2 == 0:
        df_sel = df_both[df_both["Event_Name_dictionary"].apply(lambda x: event_type in x )]
    else:
        df_sel = df_both[df_both["Event_Name_dictionary"].apply(lambda x: event_type not in x and "_".join(set(x))!='Unknown' )]
    counts, _ = np.histogram(df_sel[f"{event_type}_similarity"].dropna(), bins=bin_edges)

    plot_axis(counts, i, ax, column, num_bins, bin_edges)

# Add title and x label
axes[0].set_title("1D Heatmap Histograms of Similarities")
axes[-1].set_xlabel("Similarities with Corresponding Event Types")

plt.tight_layout()
plt.show()


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns


def get_pd_curves(df, cat='Lemma', meas=None, label=""):
    return None

    # Prepare unique lemmas
    cats = df[cat].value_counts().reset_index()[cat][:20]
    print(df[cat].value_counts())
    # Set up the plot
    plt.figure(figsize=(10, 6))

    # Optional: custom color palette
    palette = sns.color_palette("husl", len(cats))

    # Plot individual lemma distributions
    for i, lemma in enumerate(cats):
        subset = df[df[cat] == lemma]
        sns.kdeplot(
            subset[meas].dropna(),
            bw_adjust=1,
            label=lemma,
            color=palette[i],
            fill=False
        )

    # 🔁 Plot combined curve (all cats together)
    # sns.kdeplot(
    #     df[meas].dropna(),
    #     bw_adjust=1,
    #     label=f"All {cat}s",
    #     color="black",
    #     linestyle="--",
    #     linewidth=2,
    #     fill=False
    # )
    
    plt.figure()
    sns.histplot(
        df[meas].dropna(),
        label=f"All {cat}s",
        bins=15
    )

    # Customize plot
    plt.xlabel(meas)
    plt.ylabel('Density')
    plt.title(f'{meas} Distributions by {cat}')
    plt.legend(title=cat, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig(f"../exports/images/event_specific/{label}_event_type_{meas}.png")
    plt.show()
    print(df["Lemma"].value_counts())
import os
os.makedirs("../exports/images/event_specific/", exist_ok=True)
df_both_w_ET = {}
df_both_wo_ET = {}
# Explode and filter
def weight_inverted_gaussian(x, k=10):
    return 1 - np.exp(-k * (x - 0.5)**2)
for col in ['Event_Name_dictionary', 'Keyword', 'Lemma']:
    df_both[f"{col}_bkp"] = df_both[col].copy()
     
for ET in similarity_columns:
    df_both_w_ET[ET] = df_both[df_both.Event_Name_dictionary.apply(lambda x: ET in x)].drop_duplicates("Sentence_dictionary")
    df_both_wo_ET[ET] = df_both[df_both.Event_Name_dictionary.apply(lambda x: ET not in x)].drop_duplicates("Sentence_dictionary")

    df_both_w_ET[ET] = df_both_w_ET[ET].explode(['Event_Name_dictionary', 'Keyword', 'Lemma'])
    df_both_w_ET[ET] = df_both_w_ET[ET][df_both_w_ET[ET].Event_Name_dictionary==ET]
    

    df_both_wo_ET[ET] = df_both_wo_ET[ET].explode(['Event_Name_dictionary', 'Keyword', 'Lemma'])
    df_both_wo_ET[ET]['Lemma'] = df_both_wo_ET[ET]['Lemma'].apply(lambda x: x if x!='' else "<Unknown>")


    get_pd_curves(df_both_w_ET[ET],'Lemma',f'{ET}_similarity', label="With_Event_Before")
    get_pd_curves(df_both_wo_ET[ET],'Lemma',f'{ET}_similarity', label="Without_Event_Before")



    # Step 2: Add weight column
    df_both_w_ET[ET] = df_both_w_ET[ET].copy()
    df_both_w_ET[ET]["sampling_weight"] = df_both_w_ET[ET][f'{ET}_similarity'].apply(weight_inverted_gaussian)
    df_both_wo_ET[ET] = df_both_wo_ET[ET].copy()
    df_both_wo_ET[ET]["sampling_weight"] = df_both_wo_ET[ET][f'{ET}_similarity'].apply(weight_inverted_gaussian)



In [ ]:
display(df_both_w_ET[event_types[0]].Lemma.value_counts())

In [ ]:
import pandas as pd
import numpy as np

samples_per_bin_per_lemma_w_ET = 6
samples_per_bin_per_lemma_wo_ET = 1
samples_per_bin_per_unknown_lemma_w_ET = 6

import pandas as pd
import numpy as np

def bin_and_sample(df, similarity_col, lemma_col='Lemma', bins=5, samples_per_bin=6):
    global samples_per_bin_per_unknown_lemma_w_ET
    sampled_dfs = []
    bin_edges = {}
    for lemma, group in df.groupby(lemma_col):
        group = group.copy()
        # Compute consistent bins and store edges
        bin_edges[lemma] = np.histogram_bin_edges(group[similarity_col], bins=bins)
        # Use same edges in pd.cut
        group['bin'] = pd.cut(group[similarity_col], bins=bin_edges[lemma], labels=False, include_lowest=True)
        for b in range(len(bin_edges[lemma]) - 1):
            bin_group = group[group['bin'] == b]
            if not bin_group.empty:
                if lemma == "<Unknown>":
                    samples_per_bin=samples_per_bin_per_unknown_lemma_w_ET
                sample_size = min(samples_per_bin, len(bin_group))
                sampled = bin_group.sample(sample_size, random_state=42)
                sampled_dfs.append(sampled)
    sampled_df = pd.concat(sampled_dfs).drop(columns='bin')
    return sampled_df

# Step 3: Sample from each Lemma group
for ET in similarity_columns:
    kw_present_sampled_df = (
    df_both_w_ET[ET]
    .groupby("Lemma", group_keys=False)
    .apply(lambda g: bin_and_sample(
        df=g, similarity_col=f"{ET}_similarity",
        lemma_col = "Lemma", bins=5,
        samples_per_bin=samples_per_bin_per_lemma_w_ET
    )))
    kw_not_present_sampled_df = (
        df_both_wo_ET[ET]
        .groupby("Lemma", group_keys=False)
        .apply(lambda g: bin_and_sample(
            df=g, similarity_col=f"{ET}_similarity",
            lemma_col = "Lemma", bins=5,
            samples_per_bin=samples_per_bin_per_lemma_wo_ET
    )))
    
    # get_pd_curves(kw_present_sampled_df,'Lemma',f'{ET}_similarity')
    # get_pd_curves(kw_not_present_sampled_df,'Lemma',f'{ET}_similarity')

    get_pd_curves(kw_present_sampled_df,'Lemma',f'{ET}_similarity', label="With_Event_After")
    get_pd_curves(kw_not_present_sampled_df,'Lemma',f'{ET}_similarity', label="Without_Event_After")
    import os
    os.makedirs("../exports/groundtruth/F-SET/Generated", exist_ok=True)
    kw_present_sampled_df['is_keyword_present'] = True
    
    print(kw_present_sampled_df.Lemma.value_counts())
    print(kw_not_present_sampled_df.Lemma.value_counts())
    kw_not_present_sampled_df['is_keyword_present'] = False
    combined_kw_present_not_present_sampled_df = pd.concat([kw_present_sampled_df,kw_not_present_sampled_df], ignore_index=True)
    combined_kw_present_not_present_sampled_df[f"gt_{ET}"] = False
    # assert(len(combined_kw_present_not_present_sampled_df.UID)==len(combined_kw_present_not_present_sampled_df.UID).unique())
    combined_kw_present_not_present_sampled_df.to_pickle(f"../exports/groundtruth/F-SET/Generated/{ET}_{len(kw_present_sampled_df)}_{len(kw_not_present_sampled_df)}.pkl")
    combined_kw_present_not_present_sampled_df.to_excel(f"../exports/groundtruth/F-SET/Generated/{ET}_{len(kw_present_sampled_df)}_{len(kw_not_present_sampled_df)}.xlsx", index=False)



In [ ]:
for ET in event_types:
    display(ET, df_both.Event_Name_dictionary.apply(lambda x: ET in x).sum())
    
for ET in event_types:
    display( len(df_both_w_ET[ET]), df_both_w_ET[ET].Lemma.value_counts().reset_index(), kw_present_sampled_df.Lemma.value_counts())